<img src="Images/Cover.png">

# Terraforming Mars
### Data Engineering Capstone Project

#### Project Summary
The United Nations have decided to terraform Mars in the next 1500 years. So the analysts at ISRO, NASA, UN and ESDA have joined hands and have come up with an atmosphere forecast of almost 1 million Sols (i.e. sol ~ martian day) into the future. The plan starts in 2030 and they have come up with an activity schedule of the first 50,000 sols. 

They would like to open this plan and dataset to the world for further analysis and improvements. My job is to design a pipeline that can injest, structure and ready the data for the world to access.  

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 0
#### Execution Instructions
Please open AWS_CONFIG.cfg and configure your KEY, SECRET, REDSHIFT_ENDPOINT, OUTPUT_BUCKET and other REDSHIFT DWH parameters. All steps in the pipeline refer to these values.


In [1]:
# Do all imports and installs here
import pandas as pd
import configparser
import boto3
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, dayofmonth, hour, weekofyear
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import when, from_unixtime, expr, unix_timestamp, monotonically_increasing_id, to_timestamp
import os
from sql_queries import create_table_queries, drop_table_queries, copy_table_queries, insert_table_queries
import psycopg2

In [2]:
config = configparser.ConfigParser()
config.read_file(open('AWS_CONFIG.cfg'))

In [3]:
# Kindly enter the AWS Creds in the AWS_CONFIG.cfg file
# Setting the AWS credentials variable
KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')
HOST                   = config.get('AWS', 'REDSHIFT_ENDPOINT')

# Hardware configurations of the AWS Redshift Cluster
DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

# Login credentials of the AWS Redshift Cluster
DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

# Role details
DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

os.environ['AWS_ACCESS_KEY_ID'] = KEY
os.environ['AWS_SECRET_ACCESS_KEY'] = SECRET

# The input data is also placed in the S3 Bucket mentioned below & output S3 bucket location for staging
input_data = config['S3']['INPUT_BUCKET']
output_data = config['S3']['OUTPUT_BUCKET']

In [4]:
pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhCluster
4,DWH_DB,dev
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole


### Step 1: Scope the Project and Gather Data

#### Scope
Mars can be terraformed in 3 simple steps (well not so simple):
* Step 1: We need atmosphere. Martian poles have water. Nuke 'em!
* Step 2: We need oxygen. Seed creatures.
* Step 3: Then, we need heat and water. Find CO2.
<br><i>Source:https://www.vice.com/en/article/d7yz4q/a-step-by-step-guide-to-terraforming-mars</i>

Our project shows the world the activites (i.e. Nuclear Bombs, O2 Seeding etc.) we should take to terraform mars and what we can expect out of doing that (i.e. temperature, pressure etc.).

Our Project Goals:
* Get the data from the analysis teams' dump and structure for use.
* Let the open source community analyse the current plan via adhoc queries.
* Let the open source community access the data dump for plan improvements.

#### Describe and Gather Data 
We mainly have 2 datasets:
* <b>Terraforming Requirements.csv</b> contains the forecast for the next 1M Sols, information like temperature and pressure expectations of Mars in the future.
* <b>Activity Data.json</b> contains the initial activity plan of the first 50,000 Sols, activities include Nuclear Bombardments, CO2 & O2 seeding activities which are initiated from Earth. 

The data is purely synthetic with strong scientific references.

#### The Data Synthesis Phase:
* <b>Terraforming Requirements.csv</b> : https://data.world/the-pudding/mars-weather contains real martian weather data from the curiosity rover. This data spans over 2 martian years and contains no trend (because the martian atmosphere is non existent - climate hasn't changed much over time). Repeating this data for the next 1500 martian years and adding a linear trend yields a naive projection of a terraforming mars. <b>NOTEBOOK ATTACHED - Data Synthesis References/1 Terraforming Mars Data Synthesis - Terraforming Requirements.ipynb</b>.
<br/>
<br>
* <b>Activity Data.json</b> : Steps to terraform were taken from this article (https://www.vice.com/en/article/d7yz4q/a-step-by-step-guide-to-terraforming-mars) and synthetic data was generated from imaginary concepts. <b>NOTEBOOK ATTACHED - Data Synthesis References/2 Terraforming Mars Data Synthesis - Activity Data</b>
* Data dictionaries are included in the Terraforming Mars Datasets folder

In [5]:
# Reading the atmosphere projection data here
df_atmosphere = pd.read_csv('Terraforming Mars Datasets/Terraforming Requirements.csv')

In [6]:
df_atmosphere.head()

,index,ls,min_temp,max_temp,pressure,avg_temp,Sol,transformer,transformed_avg_temp,transformed_min_temp,transformed_max_temp,transformed_min_temp_dampened,transformed_max_temp_dampened,transformer_pressure,transformed_pressure
0,0,155,-76.389482,-12.134146,838.522866,-44.261814,1,-44.260000,-44.260000,-76.387668,-12.132332,-76.387636,-12.132364,838.520000,838.520000
1,1,155,-75.000000,-16.000000,739.000000,-45.500000,2,-44.259928,-45.498114,-74.998114,-15.998114,-74.998055,-15.998172,838.620487,739.097621
2,2,156,-76.000000,-11.000000,740.000000,-43.500000,3,-44.259855,-43.498041,-75.998041,-10.998041,-75.997944,-10.998138,838.720973,740.198107
3,3,156,-76.000000,-18.000000,741.000000,-47.000000,4,-44.259783,-46.997969,-75.997969,-17.997969,-75.997854,-17.998085,838.821460,741.298594
4,4,157,-74.000000,-15.000000,732.000000,-44.500000,5,-44.259711,-44.497897,-73.997897,-14.997897,-73.997750,-14.998044,838.921946,732.399080


In [7]:
df_activity = pd.read_json('Terraforming Mars Datasets/Activity Data.json')

In [8]:
df_activity.head()

,activity,activity_handler,activity_type,earth_date,ls,mars_satellite_processing_delay_mins,martian_activity_location,sol,target_pressure,target_temp,trigger_schedule_time
0,Nuclear Bomb Seed,ESDA,Level 3,1893456000000,155,2,Eberswalde,1,838.520000,-44.260000,12 AM
1,Launch Probe,United Nations,Level 2,1893542400000,155,4,Mawrth Vallis,2,739.097621,-45.498114,12 AM
2,Radiation Cleaning,ISRO,Level 1,1893628800000,156,5,Mawrth Vallis,3,740.198107,-43.498041,12 PM
3,Radiation Cleaning,ESDA,Level 1,1893715200000,156,3,"Columbia Hills, Gusev",4,741.298594,-46.997969,12 AM
4,Solar Focus,ISRO,Level 2,1893801600000,157,3,Mawrth Vallis,5,732.399080,-44.497897,12 PM


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
* Correcting nomenclature.
* Correcting datatypes.
* Dropping unncessary columns.
* Converting timestamps.
* Accounting for timestamp delays and calculating final values.

In [10]:
# Creating the spark session
'''
Creates the spark session object
'''
def create_spark_session():
    '''
    This function creates a spark session or finds an existing one and returns it
    Parameters - none
    Returns - spark session object
    '''
    
    spark = SparkSession \
                    .builder \
                    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
                    .config("spark.hadoop.fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem") \
                    .getOrCreate()
    return spark

spark = create_spark_session()

In [12]:
# Cleaning and structuring the Atmosphere projection data -> Staging the data into S3 as CSV
df_atmosphere = None
def process_atmosphere_data(spark, input_data, output_data):
    '''
    This function cleans the Atmosphere data from the csv files in S3, selects columns, transforms stores O/P as parquet in S3.
    Parameters - spark: spark session object, input_data: Path to S3 of I/P Data, output_data: Path to S3 of O/P data.
    Returns - none
    '''
    
    # get filepath to atmosphere projection data file
    atmosphere_data = input_data + 'Terraforming Requirements.csv'
    
    # read projection data file on S3
    df = spark.read.option("header","true").csv(atmosphere_data)
    df.printSchema()
    
    # Cleaning the unncesseary internal variables used for calculations
    # Assigning proper variable names
    df = df \
                .drop('index', 'transformer', 'transformed_min_temp', 'transformed_max_temp, transformer_pressure') \
                .select(
                              col('Sol').cast(IntegerType()).alias('martian_sol')
                            , col('ls').cast(DoubleType()).alias('mars_sun_angle')
                            , col('min_temp').cast(DoubleType()).alias('default_min_temp')
                            , col('avg_temp').cast(DoubleType()).alias('default_avg_temp')
                            , col('max_temp').cast(DoubleType()).alias('default_max_temp')
                            , col('transformed_min_temp_dampened').cast(DoubleType()).alias('terraformed_min_temp')
                            , col('transformed_avg_temp').cast(DoubleType()).alias('terraformed_avg_temp')
                            , col('transformed_max_temp_dampened').cast(DoubleType()).alias('terraformed_max_temp')
                            , col('pressure').cast(DoubleType()).alias('default_atm_pressure_pascal')
                            , col('transformed_pressure').cast(DoubleType()).alias('terraformed_atm_pressure_pascal')
                       )
    df.printSchema()
    
    df \
                .write \
                .mode("overwrite") \
                .csv(output_data + 'atmosphere-data/atmosphere-forecast.csv')
    
    global df_atmosphere
    df_atmosphere = df

process_atmosphere_data(spark, input_data, output_data)

root
 |-- index: string (nullable = true)
 |-- ls: string (nullable = true)
 |-- min_temp: string (nullable = true)
 |-- max_temp: string (nullable = true)
 |-- pressure: string (nullable = true)
 |-- avg_temp: string (nullable = true)
 |-- Sol: string (nullable = true)
 |-- transformer: string (nullable = true)
 |-- transformed_avg_temp: string (nullable = true)
 |-- transformed_min_temp: string (nullable = true)
 |-- transformed_max_temp: string (nullable = true)
 |-- transformed_min_temp_dampened: string (nullable = true)
 |-- transformed_max_temp_dampened: string (nullable = true)
 |-- transformer_pressure: string (nullable = true)
 |-- transformed_pressure: string (nullable = true)

root
 |-- martian_sol: integer (nullable = true)
 |-- mars_sun_angle: double (nullable = true)
 |-- default_min_temp: double (nullable = true)
 |-- default_avg_temp: double (nullable = true)
 |-- default_max_temp: double (nullable = true)
 |-- terraformed_min_temp: double (nullable = true)
 |-- terrafo

In [13]:
# Cleaning and structuring the Activity Plan data -> Staging the data into S3 as CSV
df_activity = None
def process_activity_data(spark, input_data, output_data):
    '''
    This function cleans the Activity data from the csv files in S3, selects columns, transforms stores O/P as parquet in S3.
    Parameters - spark: spark session object, input_data: Path to S3 of I/P Data, output_data: Path to S3 of O/P data.
    Returns - none
    '''
    
    # get filepath to atmosphere projection data file
    activity_data = input_data + 'Activity Data.json'
    
    # read projection data file on S3
    df = spark.read.json(activity_data)
    df.printSchema()
    
    # Cleaning the unncesseary internal variables used for calculations
    # Assigning proper variable names
    df = df \
                        .drop('ls', 'target_pressure', 'taregt_temp') \
                        .withColumn('intimation_datetime', when(df.trigger_schedule_time == '12 AM', from_unixtime(df.earth_date/1000)).otherwise(from_unixtime(df.earth_date/1000) + expr('INTERVAL 12 HOURS')))
    
    # execution time = earth start + 12 min (earth to mars light distance) + satellite_processing_delay
    df = df \
                        .withColumn('execution_datetime', unix_timestamp(df.intimation_datetime) + (df.mars_satellite_processing_delay_mins * 60) + 720) \
                        .drop('earth_date', 'mars_satellite_processing_delay_mins', 'trigger_schedule_time', 'target_temp', 'intimation_datetime')
                
    df.printSchema()

    df \
                .write \
                .mode("overwrite") \
                .csv(output_data + 'schedule-data/schedule.csv')
    
    
    global df_activity
    df_activity = df

process_activity_data(spark, input_data, output_data)

root
 |-- activity: string (nullable = true)
 |-- activity_handler: string (nullable = true)
 |-- activity_type: string (nullable = true)
 |-- earth_date: long (nullable = true)
 |-- ls: long (nullable = true)
 |-- mars_satellite_processing_delay_mins: long (nullable = true)
 |-- martian_activity_location: string (nullable = true)
 |-- sol: long (nullable = true)
 |-- target_pressure: double (nullable = true)
 |-- target_temp: double (nullable = true)
 |-- trigger_schedule_time: string (nullable = true)

root
 |-- activity: string (nullable = true)
 |-- activity_handler: string (nullable = true)
 |-- activity_type: string (nullable = true)
 |-- martian_activity_location: string (nullable = true)
 |-- sol: long (nullable = true)
 |-- execution_datetime: long (nullable = true)



### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model
<img src="Images/Conceptual DW Schema.png">

Reasons for choosing this:
* Easy for the analysts to understand.
* Welcomes ad-hoc queries.
* In case of volume increase, redundancy will be easier to maintain.

#### 3.2 Mapping Out Data Pipeline
List the steps necessary to pipeline the data into the chosen data model
<img src="Images/Pipeline.png">

Pipeline points:
* Raw data is stored in S3.
* Data is taken for cleaning and processing to a Spark Cluster
* The Spark Cluster then puts the data back in S3.
* Data is pulled by Redshift and models are created.
* A Postgres interface is exposed to the user.
* Data is also kept public in S3 bucket for bulk analysis.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [30]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(config['AWS']['REDSHIFT_ENDPOINT'], DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT))
cur = conn.cursor()

def queryExecutor(queries, cur, conn):
    for query in queries:
        cur.execute(query)
        conn.commit()
        print('Query {}.. executed.\n'.format(query[:100]))

# Executing table dropping queries
queryExecutor(drop_table_queries, cur, conn)

# Executing table creation queries
queryExecutor(create_table_queries, cur, conn)

# Executing staging queries
queryExecutor(copy_table_queries, cur, conn)

# Executing data population queries
queryExecutor(insert_table_queries, cur, conn)

Query 
    DROP TABLE IF EXISTS STAGING_ATMOSPHERE;
    DROP TABLE IF EXISTS STAGING_SCHEDULE;
    DROP TA.. executed.

Query 
        CREATE TABLE STAGING_ATMOSPHERE (
          MARTIAN_SOL INT NOT NULL
        , MARS_SUN_ANG.. executed.

Query 
    CREATE TABLE FACT_TERRAFORMANCE (
          TERRAFORMING_STEP_ID INT IDENTITY(0,1)
        , AC.. executed.

Query 
    CREATE TABLE DIM_ACTIVITIES (
          ACTIVITY_ID INT IDENTITY(0,1)
        , ACTIVITY_NAME V.. executed.

Query 
    COPY STAGING_ATMOSPHERE
    FROM 's3://terraforming-mars-staging/atmosphere-data/atmosphere-for.. executed.

Query 
    COPY STAGING_SCHEDULE
    FROM 's3://terraforming-mars-staging/schedule-data/schedule.csv'
    .. executed.

Query 

    -- ACTIVITIES
    INSERT INTO DIM_ACTIVITIES (ACTIVITY_NAME, ACTIVITY_TYPE)
    SELECT DISTINC.. executed.

Query 
        CREATE OR REPLACE FUNCTION from_unixtime(epoch NUMERIC)
        RETURNS TIMESTAMP  AS
'impo.. executed.



#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [25]:
def rowCountExecutor(query, cur, conn):
    cur.execute(query)
    return cur.fetchone()[0]

# Source/Counts check
source_row_count = df_atmosphere.count()
destination_row_count = rowCountExecutor('SELECT COUNT(*) FROM FACT_TERRAFORMANCE', cur, conn)
assert(source_row_count == destination_row_count)


# Primary Key columns must be unique
primary_key_row_count = rowCountExecutor('SELECT COUNT(ACTIVITY_ID) FROM DIM_ACTIVITIES', cur, conn)
primary_key_distinct_row_count = rowCountExecutor('SELECT DISTINCT COUNT(ACTIVITY_ID) FROM DIM_ACTIVITIES', cur, conn)
assert(primary_key_row_count == primary_key_distinct_row_count)

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Terraformance Fact table dictionary

<table dir="ltr" style="width: 734.296875px;" border="1" cellspacing="0" cellpadding="0"><colgroup><col width="219" /></colgroup>
<tbody>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;ACTIVITY_ID&quot;}">Field Name</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;ACTIVITY_ID&quot;}">Source Table</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;ACTIVITY_ID&quot;}">Meaning</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;ACTIVITY_ID&quot;}">TERRAFORMING_STEP_ID</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;ACTIVITY_ID&quot;}">Auto Increment</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;ACTIVITY_ID&quot;}">Primary Key</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;ACTIVITY_ID&quot;}">ACTIVITY_ID</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;ACTIVITY_ID&quot;}">Activities</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;ACTIVITY_ID&quot;}">Activity ID of Activity performed</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, ORGANISATION_ID&quot;}">ORGANISATION_ID</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, ORGANISATION_ID&quot;}">Organisations</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, ORGANISATION_ID&quot;}">Organisation ID</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, LOC_ID&quot;}">LOC_ID</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, LOC_ID&quot;}">Mars_Locations</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, LOC_ID&quot;}">Location ID of Mars Location</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, SOL&quot;}">SOL</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, SOL&quot;}">Current Table</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, SOL&quot;}">Solar Day</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, ACTIVITY_EXECUTION_TIME&quot;}">ACTIVITY_EXECUTION_TIME</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, ACTIVITY_EXECUTION_TIME&quot;}">Current Table</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, ACTIVITY_EXECUTION_TIME&quot;}">Timestamp of Activity being executed</td>
</tr>
<tr style="height: 21.5px;">
<td style="width: 280px; height: 21.5px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, MARS_SUN_ANGLE&quot;}">MARS_SUN_ANGLE</td>
<td style="width: 220px; height: 21.5px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, MARS_SUN_ANGLE&quot;}">Current Table</td>
<td style="width: 453.296875px; height: 21.5px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, MARS_SUN_ANGLE&quot;}">Angle between mars and sun</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_MIN_TEMP&quot;}">DEFAULT_MIN_TEMP</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_MIN_TEMP&quot;}">Current Table</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_MIN_TEMP&quot;}">Un-terraformed Mars forecast Min Temp</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_AVG_TEMP&quot;}">DEFAULT_AVG_TEMP</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_AVG_TEMP&quot;}">Current Table</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_AVG_TEMP&quot;}">
<table dir="ltr" style="width: 734.296875px;" border="1" cellspacing="0" cellpadding="0">
<tbody>
<tr style="height: 21px;">
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_MIN_TEMP&quot;}">Un-terraformed Mars forecast Avg&nbsp;Temp</td>
</tr>
</tbody>
</table>
</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_MAX_TEMP&quot;}">DEFAULT_MAX_TEMP</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_MAX_TEMP&quot;}">Current Table</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_MAX_TEMP&quot;}">
<table dir="ltr" style="width: 734.296875px;" border="1" cellspacing="0" cellpadding="0">
<tbody>
<tr style="height: 21px;">
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_MIN_TEMP&quot;}">Un-terraformed Mars forecast Max&nbsp;Temp</td>
</tr>
</tbody>
</table>
</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_MIN_TEMP&quot;}">TERRAFORMED_MIN_TEMP</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_MIN_TEMP&quot;}">Current Table</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_MIN_TEMP&quot;}">
<table dir="ltr" style="width: 734.296875px;" border="1" cellspacing="0" cellpadding="0">
<tbody>
<tr style="height: 21px;">
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_MIN_TEMP&quot;}">Terraformed Mars forecast Min Temp</td>
</tr>
</tbody>
</table>
</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_AVG_TEMP&quot;}">TERRAFORMED_AVG_TEMP</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_AVG_TEMP&quot;}">Current Table</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_AVG_TEMP&quot;}">
<table dir="ltr" style="width: 734.296875px;" border="1" cellspacing="0" cellpadding="0">
<tbody>
<tr style="height: 21px;">
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_MIN_TEMP&quot;}">
<table dir="ltr" style="width: 734.296875px;" border="1" cellspacing="0" cellpadding="0">
<tbody>
<tr style="height: 21px;">
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_MIN_TEMP&quot;}">Terraformed Mars forecast Avg&nbsp;Temp</td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_MAX_TEMP&quot;}">TERRAFORMED_MAX_TEMP</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_MAX_TEMP&quot;}">Current Table</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_MAX_TEMP&quot;}">
<table dir="ltr" style="width: 734.296875px;" border="1" cellspacing="0" cellpadding="0">
<tbody>
<tr style="height: 21px;">
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_MIN_TEMP&quot;}">
<table dir="ltr" style="width: 734.296875px;" border="1" cellspacing="0" cellpadding="0">
<tbody>
<tr style="height: 21px;">
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_MIN_TEMP&quot;}">Terraformed Mars forecast Max&nbsp;Temp</td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_ATM_PRESSURE_PASCAL&quot;}">
<div>
<div>DEFAULT_ATM_PRESSURE_PASCAL</div>
</div>
</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_ATM_PRESSURE_PASCAL&quot;}">
<div>
<div>Current Table</div>
</div>
</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, DEFAULT_ATM_PRESSURE_PASCAL&quot;}">Un-Terraformed Mars Pressure in pascal</td>
</tr>
<tr style="height: 21px;">
<td style="width: 280px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_ATM_PRESSURE_PASCAL&quot;}">
<div>
<div>TERRAFORMED_ATM_PRESSURE_PASCAL</div>
</div>
</td>
<td style="width: 220px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_ATM_PRESSURE_PASCAL&quot;}">
<div>
<div>Current Table</div>
</div>
</td>
<td style="width: 453.296875px; height: 21px;" data-sheets-value="{&quot;1&quot;:2,&quot;2&quot;:&quot;, TERRAFORMED_ATM_PRESSURE_PASCAL&quot;}">Terraformed Mars Pressure in Pascal</td>
</tr>
</tbody>
</table>

##### Activity Dim Dictionary
<table>
<tbody>
<tr>
<td>FieldName&nbsp;</td>
<td>Source&nbsp;</td>
<td>&nbsp;Meaning</td>
</tr>
<tr>
<td>&nbsp;Activity_ID</td>
<td>&nbsp;Auto Generated</td>
<td>&nbsp;Primary Key</td>
</tr>
<tr>
<td>&nbsp;Activity_name</td>
<td>&nbsp;Staging_schedule</td>
<td>&nbsp;Name of activity</td>
</tr>
<tr>
<td>Activity_type</td>
<td>Staging_schedule</td>
<td>Level of activity</td>
</tr>
</tbody>
</table>
<!-- DivTable.com -->

##### Organisations Dim Dictionary
<table>
<tbody>
<tr>
<td>FieldName&nbsp;</td>
<td>Source&nbsp;</td>
<td>&nbsp;Meaning</td>
</tr>
<tr>
<td>&nbsp;Org_ID</td>
<td>&nbsp;Auto Generated</td>
<td>&nbsp;Primary Key</td>
</tr>
<tr>
<td>Organisation_name</td>
<td>&nbsp;Staging_schedule</td>
<td>&nbsp;Name of&nbsp;organisation handling activity</td>
</tr>
</tbody>
</table>
<!-- DivTable.com -->

##### Martian Locations
<table>
<tbody>
<tr>
<td>FieldName&nbsp;</td>
<td>Source&nbsp;</td>
<td>&nbsp;Meaning</td>
</tr>
<tr>
<td>Loc_ID</td>
<td>&nbsp;Auto Generated</td>
<td>&nbsp;Primary Key</td>
</tr>
<tr>
<td>location_name</td>
<td>&nbsp;Staging_schedule</td>
<td>&nbsp;Name of&nbsp;location&nbsp;on Mars</td>
</tr>
</tbody>
</table>
<!-- DivTable.com -->

##### Mars Time Dim Table
<table>
<tbody>
<tr style="height: 23px;">
<td style="height: 23px;">FieldName&nbsp;</td>
<td style="height: 23px;">Source&nbsp;</td>
<td style="height: 23px;">&nbsp;Meaning</td>
</tr>
<tr style="height: 23.5px;">
<td style="height: 23.5px;">Sol</td>
<td style="height: 23.5px;">Staging_atmosphere</td>
<td style="height: 23.5px;">&nbsp;Solar day, Primary Key</td>
</tr>
</tbody>
</table>
<!-- DivTable.com -->

##### Earth Time Dim Table
<table>
<tbody>
<tr style="height: 23px;">
<td style="height: 23px;">&nbsp;FieldName</td>
<td style="height: 23px;">Source&nbsp;</td>
<td style="height: 23px;">&nbsp;Meaning</td>
</tr>
<tr style="height: 23px;">
<td style="height: 23px;">&nbsp;Start_time</td>
<td style="height: 23px;">&nbsp;Staging_schedule</td>
<td style="height: 23px;">&nbsp;Activity Execution Time, Primary Key</td>
</tr>
<tr style="height: 23px;">
<td style="height: 23px;">&nbsp;Hour</td>
<td style="height: 23px;">&nbsp;Start_time</td>
<td style="height: 23px;">&nbsp;Hour</td>
</tr>
<tr style="height: 23px;">
<td style="height: 23px;">&nbsp;Day</td>
<td style="height: 23px;">&nbsp;Start_time</td>
<td style="height: 23px;">&nbsp;Day</td>
</tr>
<tr style="height: 23px;">
<td style="height: 23px;">&nbsp;Month</td>
<td style="height: 23px;">&nbsp;Start_time</td>
<td style="height: 23px;">&nbsp;Month</td>
</tr>
<tr style="height: 23.5px;">
<td style="height: 23.5px;">Year</td>
<td style="height: 23.5px;">Start_time</td>
<td style="height: 23.5px;">Year</td>
</tr>
</tbody>
</table>
<!-- DivTable.com -->

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
 * S3 - Scalable storage for large sized data.
 * Spark - Used for parallel computation and better for processing data smartly. Hence preferred for data cleansing. If this phase included intelligence tasks in the future, Spark supports those complex libraries.
 * Redshift - Scalable and SQL friendly data warehouse.
 * Airflow - Used to pipeline and schedule everything.
 
 
* Propose how often the data should be updated and why.
 * The data should be updated monthly for this dataset. Considering the nature of the problem the solution will not be very frequent. So alterations and new versions can be deployed at a monthly, at max a weekly frequency.
 
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
     * More processing power would be needed, setting up more nodes (since independant tasks are being performed on Spark in this case.
     * I would not change the other technologies of the pipeline, however I would partition the data in S3 for staging more effectively.
     
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
  * Materialised views in Redshift can be created that serve the dashboard by the Airflow pipeline.
  
 * The database needed to be accessed by 100+ people.
  * Materialised views can also serve as table redundancies which will help reads. Since we are not a write heavy player here we can focus on redundancy responsibly via Airflow pipelines.